# JP2000 USAGE

## 1 - Configuration

### 1.1 - Imports

In [ ]:
import sys
sys.path.append("../lib") 
import gdal
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np
import math
import getpass
from mundilib import MundiCatalogue, get_node
from utils import city_polygon_bbox, country_polygon_bbox
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import folium

### 1.2 - Use your acces key

In [ ]:
print("Please enter your acces key: ")
ak = input()
print("Please enter your secret key: ")
sk = input()

gdal.SetConfigOption("AWS_ACCESS_KEY_ID", ak)
gdal.SetConfigOption("AWS_SECRET_ACCESS_KEY", sk)
gdal.SetConfigOption("AWS_REGION", "eu-de")
gdal.SetConfigOption("AWS_VIRTUAL_HOSTING", "FALSE")
gdal.SetConfigOption("AWS_S3_ENDPOINT", "obs.eu-de.otc.t-systems.com")
gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.SetConfigOption("AWS_DEFAULT_REGION", "eu-de")

### 1.3 - Select Bounding Box

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

lonmin = widgets.FloatText(value = 1.40,  description = 'Long min:', disabled = False, step = 0.01)
latmin = widgets.FloatText(value = 43.55, description = 'Lat min:',  disabled = False, step = 0.01)
lonmax = widgets.FloatText(value = 1.50,  description = 'Long max:', disabled = False, step = 0.01)
latmax = widgets.FloatText(value = 43.65, description = 'Lat max:',  disabled = False, step = 0.01)

bbox_menu = widgets.VBox([lonmin, latmin, lonmax, latmax])
country_menu = widgets.Text(placeholder = 'Ex: Germany, Spain...')
cities_menu = widgets.Text(placeholder = 'Ex: Toulouse, Paris...')

children = [bbox_menu, 
            widgets.VBox([
                widgets.Label(value="Enter a country:"),
                country_menu
            ]),
            widgets.VBox([
                widgets.Label(value="Enter a city:"),
                cities_menu
            ])]

tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'Bounding Box')
tab.set_title(1, 'Country')
tab.set_title(2, 'Cities')

bbox = [1.40, 43.55, 1.50, 43.65]
out = widgets.Output()
button = widgets.Button(description = 'Validate')
vbox = widgets.VBox([button, out])

display(tab)
display(vbox)

def click(b):
    global bbox
    if (tab.selected_index == 0):
        bbox = [lonmax.value, latmax.value, lonmin.value, latmin.value]
    elif(tab.selected_index == 1):
        polygon, bbox = country_polygon_bbox(country_menu.value)
    elif(tab.selected_index == 2):
        polygon, bbox, place_name = city_polygon_bbox(cities_menu.value)
    lonmin.value = bbox[2]
    latmin.value = bbox[3]
    lonmax.value = bbox[0]
    latmax.value = bbox[1]
    with out:
        print(bbox)
button.on_click(click)

### 1.4 - Select in mundi catalog

In [ ]:
c = MundiCatalogue()
csw = c.get_collection("Sentinel2").mundi_csw()
t = datetime.utcnow() + timedelta(days = -6)
xml_string='''
<?xml version='1.0'?>
<GetRecords xmlns='http://www.opengis.net/cat/csw/2.0.2'
    xmlns:DIAS='http://tas/DIAS'
    xmlns:csw='http://www.opengis.net/cat/csw/2.0.2'
    xmlns:dc='http://purl.org/dc/elements/1.1/'
    xmlns:dct='http://purl.org/dc/terms/'
    xmlns:gml='http://www.opengis.net/gml'
    xmlns:ogc='http://www.opengis.net/ogc'
    xmlns:ows='http://www.opengis.net/ows'
    xmlns:xsi='http://www.w3.org/2001/XMLSchema-instance'
    service='CSW' version='2.0.2' maxRecords='50' startPosition='1' resultType='results'
    outputFormat='application/xml' outputSchema='csw:Record' xsi:schemaLocation='http://www.opengis.net/cat/csw/2.0.2/CSW-discovery.xsd'>
    <csw:Query typeNames='csw:Record'>
        <csw:ElementSetName>full</csw:ElementSetName>
        <csw:Constraint version='1.1.0'>
            <ogc:Filter xmlns:ogc="http://www.opengis.net/ogc">
                <csw:And>
                     <csw:Or>
                        <ogc:PropertyIsEqualTo>
                            <ogc:PropertyName>DIAS:onlineStatus</ogc:PropertyName>
                            <ogc:Literal>STAGING</ogc:Literal>
                        </ogc:PropertyIsEqualTo>
                        <ogc:PropertyIsEqualTo>
                            <ogc:PropertyName>DIAS:onlineStatus</ogc:PropertyName>
                            <ogc:Literal>ONLINE</ogc:Literal>
                        </ogc:PropertyIsEqualTo>
                    </csw:Or>
                    <ogc:PropertyIsGreaterThan>
                        <ogc:PropertyName>DIAS:sensingStartDate</ogc:PropertyName>
                        <ogc:Literal>{1}Z</ogc:Literal>
                    </ogc:PropertyIsGreaterThan>
                    <ogc:Intersects>
                        <ogc:PropertyName>DIAS:footprint</ogc:PropertyName>
                        <gml:Envelope xmlns:gml="http://www.opengis.net/gml" srsName="EPSG:4326">                        
                            <gml:lowerCorner>{0[2]} {0[3]}</gml:lowerCorner>
                            <gml:upperCorner>{0[0]} {0[1]}</gml:upperCorner>
                        </gml:Envelope>
                    </ogc:Intersects>
                 </csw:And>
            </ogc:Filter>
        </csw:Constraint>
        <ogc:SortBy xmlns:ogc="http://www.opengis.net/ogc">
            <ogc:SortProperty>
                <ogc:PropertyName>DIAS:sensingStartDate</ogc:PropertyName>
                <ogc:SortOrder>DESC</ogc:SortOrder>
            </ogc:SortProperty>
        </ogc:SortBy>
    </csw:Query>
</GetRecords>'''.format(bbox, t.isoformat(timespec='seconds'))
payload = xml_string.strip()
csw.get_records(xml = payload)
print("Number of dataset records: %d" % len(csw.records))
if(len(csw.records) == 0):
    raise(Exception("No image Found"))

product_menu = widgets.Dropdown(
    options=csw.records,
    description='Produit:',
    disabled=False,
    width='100%',
)
display(product_menu)

### 1.5 - Select a file

In [ ]:
import boto3

b = product_menu.value
baseurl = get_node(b, "DIAS:archiveProductURI").text
baseurl.replace("https://mundiwebservices.com/dp", "https://obs.eu-de.otc.t-systems.com")

client = boto3.client(
    's3',
    aws_access_key_id=ak,
    aws_secret_access_key=sk,
    endpoint_url='https://obs.eu-de.otc.t-systems.com',
)
basekey = baseurl[51:]
key = basekey + "/manifest.safe"
bucket = baseurl[36:50]
res = client.get_object(Bucket=bucket, Key=key)
tree = ET.fromstring(res["Body"].read().decode('utf-8'))
root = tree
dataobjects = root.find("dataObjectSection")
files = {}
for i in dataobjects:
    if "ID" in i.attrib and i.attrib["ID"].startswith("IMG_DATA_Band"):
        files[i[0][0].attrib["href"].split("/")[-1]] = basekey + i[0][0].attrib["href"][1:]
files_menu = widgets.Dropdown(
    options=files,
    description='File:',
    disabled=False,
    width='100%',
)
scale_menu = widgets.IntSlider(
    value=2,
    min=0,
    max=10,
    step=1,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(files_menu, scale_menu)

## 2 - Using images

In [ ]:
my_image = gdal.Open('/vsis3/'+ bucket +"/"+files_menu.value)
scale = scale_menu.value
band1 = my_image.GetRasterBand(1)
if scale > 0 and band1.GetOverviewCount() > scale:
    band1 = my_image.GetRasterBand(1).GetOverview(scale)
print(band1.XSize, band1.YSize)

## 3.3 - Compute localisation

In [ ]:
gt = my_image.GetGeoTransform()
scale = 0
band1 = my_image.GetRasterBand(1)
if scale > 0 and band1.GetOverviewCount() > scale:
    band1 = my_image.GetRasterBand(1).GetOverview(scale)

gt = [i for i in gt]
gt[1], gt[5] = (gt[1],gt[5]) if scale == 0 else (gt[1] * 2 ** (scale + 1), (gt[5] * 2 ** (scale + 1)))
print(gt)

In [ ]:
from osgeo import ogr
from osgeo import osr
proj = osr.SpatialReference(wkt=my_image.GetProjection())

source = osr.SpatialReference()
source.ImportFromEPSG(4326)

target = osr.SpatialReference()
target.ImportFromEPSG(int(proj.GetAttrValue('AUTHORITY',1)))

transform = osr.CoordinateTransformation(source, target)
point = ogr.Geometry(ogr.wkbLinearRing)
point.AddPoint(bbox[0], bbox[1])
point.AddPoint(bbox[2], bbox[3])
print(point)
point.Transform(transform)

bbox = [point.GetPoint(0)[0], point.GetPoint(0)[1], point.GetPoint(1)[0], point.GetPoint(1)[1]]

In [ ]:
mx1 = bbox[2] 
my1 = bbox[3]
mx2 = bbox[0]
my2 = bbox[1]
px1 = int((mx1 - gt[0]) / gt[1])
py1 = int((my1 - gt[3]) / gt[5])
px2 = int((mx2 - gt[0]) / gt[1])
py2 = int((my2 - gt[3]) / gt[5])
position = [min(px1, px2), min(py1, py2), max(px1, px2), max(py1, py2)]
size = [position[2] - position[0], position[3] - position[1]]

k = np.frombuffer(band1.ReadRaster1(position[0], position[1], size[0], size[1]),
                  dtype='uint16', count = size[0] * size[1]).reshape([size[1], size[0]])

left = gt[0] + position[0]*gt[1]
top = gt[3] + position[1]*gt[5]
subset_extent = (left, left + size[0]*gt[1], 
                   top + size[1]*gt[5], top)
print(subset_extent)

In [ ]:
transform = osr.CoordinateTransformation(target, source)
point = ogr.Geometry(ogr.wkbLinearRing)
point.AddPoint(subset_extent[0], subset_extent[2])
point.AddPoint(subset_extent[1], subset_extent[3])
print(point)
point.Transform(transform)
print(point)
subset_extent = [point.GetPoint(0)[0], point.GetPoint(1)[0], point.GetPoint(0)[1], point.GetPoint(1)[1]]

## 3.4 - Show on map

In [ ]:
m = folium.Map(location=(subset_extent[2], subset_extent[0]))
def colmap(x):
    if x != 0:
        temp = np.log(x)
        return (temp, temp, temp, 1) 
    else:
        return (0, 0, 0, 0)
folium.raster_layers.ImageOverlay(
    image=k,
    bounds=[(subset_extent[2], subset_extent[0]), (subset_extent[3], subset_extent[1])],
    origin='upper',
    opacity=1
).add_to(m)
folium.vector_layers.Polygon(
    locations=[(subset_extent[2], subset_extent[0]), (subset_extent[3], subset_extent[0]), (subset_extent[3], subset_extent[1]), (subset_extent[2], subset_extent[1])],
    color="lightblue",
).add_to(m)
folium.LayerControl().add_to(m)
m.save('/home/jovyan/work/folium_map.html')